<a href="https://colab.research.google.com/github/lisstasy/tenderAI/blob/main/TenderAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken chromadb sentence-transformers jq langchain_openai unstructured pypdf langchain_community

In [ ]:
from langchain.document_loaders import JSONLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader, UnstructuredExcelLoader, PyPDFLoader
from langchain.prompts import ChatPromptTemplate

from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

openai_api_key = "###"

In [ ]:
def load_xlsl():

    loader = UnstructuredExcelLoader('/content/3t_TZ-TEST.xlsx') #, mode="elements"
    docs_xls = loader.load()
    return docs_xls

doc=load_xlsl()

In [ ]:
def load_pdf():

    loader = PyPDFLoader("/content/1.5_TZ.pdf")
    pages = loader.load()

    return pages

doc = load_pdf()

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)
db_for_temporal_data = Chroma.from_documents(doc, embedding_function)
retriever_temp = db_for_temporal_data.as_retriever()

template = """Extract from the {context} key and values. Possiable keys:value pair are capacity_kg,service_life_years,lifting_speed_mm_s,engine_type,lifting_height_mm,dimensions_mm. Be attentive for capacity_kg, if you see value "1.5" it means 1500kg.

Instructions: {instructions}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model ="gpt-4-0125-preview", openai_api_key=openai_api_key)

chain = (
    {"context": retriever_temp, "instructions": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = "extract in format of the table in English"

# Invoke the chain and save the output
output = chain.invoke(query)

# Now you can use 'output' as an input in the next chain
next_chain_input = output



In [ ]:
output

'| Key                | Value                  |\n|--------------------|------------------------|\n| capacity_kg        | 3000                   |\n| service_life_years | 5                      |\n| lifting_speed_mm_s | Not available          |\n| engine_type        | Diesel (Japan), Isuzu C240 (Japan) |\n| lifting_height_mm  | Not less than 7000     |\n| dimensions_mm      | Not available          |'

In [ ]:
print(type(next_chain_input))

<class 'str'>


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

file_paths = ["/content/1.5T_WEB(real)_mod.json", "/content/3.5TO_WEB_mod.json", "/content/3TO_WEB_mod.json", "/content/3T_WEB(real).json"]

# Load documents from all files
all_documents = []
for file_path in file_paths:
    loader = JSONLoader(file_path=file_path, jq_schema=".data", text_content=False)
    documents = loader.load()
    all_documents.extend(documents)


# save to disk
db = Chroma.from_documents(all_documents, embedding_function, persist_directory="./chroma_db", collection_name="langchain")
retriever = db.as_retriever()


In [ ]:
# Define the template for the prompt
template = """Given the specifications {next_chain_input}, answer the question using the following context to help: {context}.

Instruction: {instruction}
"""

prompt = ChatPromptTemplate.from_template(template)
next_chain_input
# Define the model
model = ChatOpenAI(model="gpt-4-0125-preview", openai_api_key=openai_api_key) #gpt-3.5-turbo-0125 gpt-4-0125-preview

# Define the processing chain
chain = (
    {"context": retriever, "instruction": RunnablePassthrough(), "next_chain_input": RunnablePassthrough()} #next_chain_input #RunnablePassthrough() retriever_temp
    | prompt
    | model
    | StrOutputParser()
)

# Example query
instruction = "You are an expirienced sales person. Write a commercial offer in Russian that proposes the machinery matching the spesifications the most. Include in your offer key data such as: capacity_kg,service_life_years,lifting_speed_mm_s,engine_type,lifting_height_mm,dimensions_mm."

print(chain.invoke(instruction))

Уважаемые господа!

Рады предложить вам наш новейший дизельный погрузчик Hangcha CPCD30-XRW10, который является идеальным решением для ваших складских и производственных нужд. Этот погрузчик сочетает в себе высокую производительность, экономичность эксплуатации и удобство использования.

**Основные характеристики:**

- **Грузоподъемность:** 3000 кг, что позволяет с легкостью справляться с большинством задач на складе или в производственной зоне.
- **Срок службы:** Наш погрузчик рассчитан на длительный срок службы, до 10 лет, обеспечивая вашему бизнесу надежного "рабочего коня" на годы вперед.
- **Скорость подъема:** 0.49/0.53 м/с (с грузом/без груза) обеспечивает быструю и эффективную работу с грузами, что значительно ускоряет процессы на складе.
- **Тип двигателя:** Дизельный, что гарантирует высокую мощность и экономичность в эксплуатации по сравнению с бензиновыми аналогами.
- **Высота подъема:** до 7000 мм, позволяющая работать с грузами на большой высоте.
- **Размеры:** 2700мм (дл